In [1]:
!pip install torch wandb datasets \
transformers peft trl \
bitsandbytes qwen-vl-utils


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import wandb
from datasets import load_dataset, load_from_disk
from transformers import (
    Qwen2_5_VLForConditionalGeneration,
    AutoProcessor,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer
from qwen_vl_utils import process_vision_info


2025-09-19 08:55:25.700805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758272126.054886      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758272126.164477      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Carica il dataset HuggingFace
dataset_id = "/kaggle/input/dataset-fixed-path-2/kaggle/working/fine-tuning-dataset-fixed_2"  # Cambia con il tuo dataset
dataset = load_from_disk(dataset_id)
print(dataset)

Dataset({
    features: ['img_id', 'image', 'image_k', 'image_ssh', 'label', 'x_t', 'label_str', 'prompt_static'],
    num_rows: 1001
})


In [4]:
print(dataset[0]['image_ssh'])

/mnt/hhd1/maffongelli/new/new_data/real/auto_factory_3.jpg


In [4]:
import shutil

# Copia il dataset in working
src_path = "/kaggle/input/finetuning-dataset/dataset2_finetuning"
dst_path = "/kaggle/working/dataset2_finetuning_2"
shutil.copytree(src_path, dst_path)

dataset_fix = load_from_disk(dst_path)

# Correggere il path in image_k per renderlo assoluto
def fix_image_path(sample):
    # Se il tuo path attuale è 'kaggle/input/...', aggiungi lo slash iniziale
    if sample["image_k"].startswith("kaggle/input/"):
        sample["image_k"] = "/" + sample["image_k"]
    return sample

# Applicalo ai dataset divisi
dataset_fix = dataset_fix.map(fix_image_path)
dataset_fix.save_to_disk("/kaggle/working/fine-tuning-dataset-fixed_2")

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1001 [00:00<?, ? examples/s]

In [22]:
!zip -r /kaggle/working/fine-tuning-dataset-fixed_2.zip /kaggle/working/fine-tuning-dataset-fixed_2


  adding: kaggle/working/fine-tuning-dataset-fixed_2/ (stored 0%)
  adding: kaggle/working/fine-tuning-dataset-fixed_2/dataset_info.json (deflated 76%)
  adding: kaggle/working/fine-tuning-dataset-fixed_2/data-00000-of-00001.arrow (deflated 77%)
  adding: kaggle/working/fine-tuning-dataset-fixed_2/state.json (deflated 46%)


In [3]:
from PIL import Image
import os
def convert_to_required_format(sample):
    """
    Convert a single dataset sample into the required format.
    """
    # Carica l'immagine dal percorso
    image_path = sample["image_k"]
    image = Image.open(image_path).convert("RGB")  # Carica l'immagine e convertila in RGB
    
    # Crea la struttura del dato formattato
    return {
        "texts": {
            "system": "You are an AI assistant specialized in detecting whether an image is real or digitally manipulated.",
            "user": sample["prompt_static"],  # Il prompt statico
            "assistant": sample["label_str"],  # La risposta dell'assistente (REAL/FAKE)
        },
        "images": image  # Percorso dell'immagine o PIL.Image
    }

# Carica il dataset HuggingFace
dataset_id = "/kaggle/input/dataset-fixed-path-2/kaggle/working/fine-tuning-dataset-fixed_2"  # Cambia con il tuo dataset
dataset = load_from_disk(dataset_id)

# Adatta il dataset
dataset = [convert_to_required_format(sample) for sample in dataset]

# Verifica la struttura del primo sample formattato
print(dataset[0])


{'texts': {'system': 'You are an AI assistant specialized in detecting whether an image is real or digitally manipulated.', 'user': 'Is the image real or fake? Choose one: [REAL] / [FAKE]', 'assistant': 'FAKE'}, 'images': <PIL.Image.Image image mode=RGB size=400x247 at 0x7E4A95D6DCD0>}


In [4]:
print(dataset[:5])

[{'texts': {'system': 'You are an AI assistant specialized in detecting whether an image is real or digitally manipulated.', 'user': 'Is the image real or fake? Choose one: [REAL] / [FAKE]', 'assistant': 'FAKE'}, 'images': <PIL.Image.Image image mode=RGB size=400x247 at 0x7E4A95D6DCD0>}, {'texts': {'system': 'You are an AI assistant specialized in detecting whether an image is real or digitally manipulated.', 'user': 'Is the image real or fake? Choose one: [REAL] / [FAKE]', 'assistant': 'REAL'}, 'images': <PIL.Image.Image image mode=RGB size=768x768 at 0x7E4AA4906990>}, {'texts': {'system': 'You are an AI assistant specialized in detecting whether an image is real or digitally manipulated.', 'user': 'Is the image real or fake? Choose one: [REAL] / [FAKE]', 'assistant': 'REAL'}, 'images': <PIL.Image.Image image mode=RGB size=768x768 at 0x7E4AA49DDA50>}, {'texts': {'system': 'You are an AI assistant specialized in detecting whether an image is real or digitally manipulated.', 'user': 'Is

In [6]:
def format_data(sample):
    """
    Format a single dataset sample into the required structure.
    """
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": sample["texts"]["system"]}],
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": sample["images"]},
                {"type": "text", "text": sample["texts"]["user"]},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["texts"]["assistant"]}],
        },
    ]


In [7]:
dataset = [format_data(sample) for sample in dataset]

In [8]:
print(dataset[0])

[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are an AI assistant specialized in detecting whether an image is real or digitally manipulated.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=400x247 at 0x7E4A95D6DCD0>}, {'type': 'text', 'text': 'Is the image real or fake? Choose one: [REAL] / [FAKE]'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'FAKE'}]}]


In [9]:
def collate_fn(examples):
    """
    Data collator to prepare a batch of examples.
    This function applies the chat template to texts, processes the images,
    tokenizes the inputs, and creates labels with proper masking.
    """
    # Apply chat template to each example (no tokenization here)
    texts = [processor.apply_chat_template(example, tokenize=False) for example in examples]
    
    # Process visual inputs for each example
    image_inputs = [process_vision_info(example)[0] for example in examples]

    # Tokenize texts and images into tensors with padding
    batch = processor(
        text=texts,
        images=image_inputs,
        return_tensors="pt",
        padding=True,
    )

    # Create labels by cloning input_ids and mask the pad tokens
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100

    # Determine image token IDs to mask in the labels (model specific)
    image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]

    # Mask image token IDs in the labels
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100

    batch["labels"] = labels
    return batch


In [ ]:
!pip uninstall bitsandbytes
!pip install bitsandbytes


Found existing installation: bitsandbytes 0.47.0
Uninstalling bitsandbytes-0.47.0:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/bitsandbytes-0.47.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/bitsandbytes/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [10]:
# Model and processor configuration
model_id = "Qwen/Qwen2.5-VL-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=bnb_config
)
processor = AutoProcessor.from_pretrained(model_id)


`torch_dtype` is deprecated! Use `dtype` instead!


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [11]:
# Configure LoRA for model adaptation
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

# Apply PEFT model adaptation and print trainable parameters
peft_model = get_peft_model(model, peft_config)  # Ricarica il modello con la nuova configurazione PEFT

peft_model.print_trainable_parameters()


trainable params: 1,843,200 || all params: 3,756,466,176 || trainable%: 0.0491


In [12]:
# Configure training arguments using SFTConfig
training_args = SFTConfig(
    output_dir="finetuned",  # Directory to save the model
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=1,  # Training batch size per device (puoi lasciarlo così o ridurlo ulteriormente)
    per_device_eval_batch_size=1,  # Evaluation batch size per device
    gradient_accumulation_steps=8,  # Aumenta l'accumulo di gradienti (ad esempio su 8 passaggi)
    gradient_checkpointing=True,  # Gradient checkpointing per memoria efficiente
    optim="adamw_torch_fused",  # Optimizer type
    learning_rate=2e-4,  # Learning rate for training
    lr_scheduler_type="constant",  # Learning rate scheduler type
    logging_steps=10,  # Interval (in steps) for logging
    eval_steps=10,  # Interval (in steps) for evaluation
    eval_strategy="steps",  # Evaluation strategy
    save_strategy="steps",  # Strategy for saving the model
    save_steps=20,  # Interval (in steps) for saving
    metric_for_best_model="eval_loss",  # Metric to evaluate the best model
    greater_is_better=False,  # Lower metric values are better
    load_best_model_at_end=True,  # Load the best model after training
    bf16=False,  # Disable bfloat16 precision
    tf32=False,  # Disable TensorFloat-32 precision
    max_grad_norm=0.3,  # Maximum gradient norm for clipping
    warmup_ratio=0.03,  # Warmup ratio for learning rate scheduler
    report_to="wandb",  # Reporting via Weights & Biases
    push_to_hub=False,  # Do not push the model to Hugging Face Hub
    gradient_checkpointing_kwargs={"use_reentrant": False},  # Gradient checkpointing options
    dataset_text_field="",  # Text field in the dataset (if applicable)
    dataset_kwargs={"skip_prepare_dataset": True},  # Additional dataset options
)


training_args.fp16 = True  # Usa FP16 per ridurre l'uso della memoria


# Ensure unused columns are not removed
training_args.remove_unused_columns = False


In [13]:
import os
os.environ["WANDB_API_KEY"] = "bf075f6a351670e912260855f80b46912ff79436" 


In [14]:
import wandb
wandb.login()


wandb: Currently logged in as: maffongelli-1941528 (maffongelli-1941528-la-sapienza) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [15]:
# Initialize Weights & Biases for experiment tracking
wandb.init(
    project="deepfake-detection",  # Update project name as needed
    config=training_args,
)


In [16]:
from sklearn.model_selection import train_test_split

# Splitta il dataset in training e test
train_dataset, eval_dataset = train_test_split(dataset, test_size=0.1, random_state=42)

# Verifica la dimensione dei dataset splittati
print("Training samples:", len(train_dataset))
print("Validation samples:", len(eval_dataset))


Training samples: 900
Validation samples: 101


In [17]:
# Create the trainer for fine-tuning the model
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [18]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.32 GiB. GPU 0 has a total capacity of 14.74 GiB of which 3.22 GiB is free. Process 3456 has 11.52 GiB memory in use. Of the allocated memory 9.06 GiB is allocated by PyTorch, and 2.33 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)